# Week 3: Building a stemmer and set of stop words

In this weeks notebook we are going to build a simple stemmer and a set of stop words to use for our nursery rhymes dataset. 

We will be using bag of words again to represent our nursery rhymes, though this time we will be using the [CountVectorizer class](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) from the Python library [sci-kit learn](https://scikit-learn.org/stable/index.html).

First lets do some imports:

In [42]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

This code cell defines a simple stemmer using [regex](https://www.w3schools.com/python/python_regex.asp). The code that defines the rules of the stemmer is commented out so it does not initially do anything, but we will come back to this later in the tasks, when we start building our stemmer.

The first rule `(r's$', '')` looks to find the character `s` at the end of a word (which we define with `$`) and replaces it with an empty string `''`. When we uncomment this then any word with an `s` at the end with have that letter removed. The goal of this is to make words that are plural into  [singular nouns](https://www.ef.com/wwen/english-resources/english-grammar/singular-and-plural-nouns/). 

For now leave this as it is and run this cell, we will come back to this cell when we start to build our stemmer.

<a id='stemmer'></a>

In [43]:
def simple_stemmer(word):
    # Define a list of stemming rules as tuples (pattern, replacement)
    stemming_rules = [
    # Uncomment the line below and add more rules when you come to Task 2
        # (r's\b', ''),    
    ]
    
    # Apply each rule to the word
    for pattern, replacement in stemming_rules:
        word = re.sub(pattern, replacement, word)
    
    return word

This fuction will go through every text file in a folder and load the contents of the file to the list `document_texts` and the name of the file to the list `documents_labels`. As you can see before it puts the text into the `documents_text` list, it first passes all of the words through the `simple_stemmer` function that we have just defined:

In [44]:
def load_text_documents(folder_path):
    # Make empty arrays for documents and file names
    document_texts = []
    document_labels = []

    # Find the files in the directory
    for root, _, files in os.walk(folder_path):
        
        # Go through each file 
        for file in files:
            
            # If it is a text file, read it and add it to the list of docuemnts
            # along with the filename (minus the extension)
            if file.endswith(".txt"):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    text = f.read()
                
                # Apply stemming to each word in the nursery rhyme
                stemmed_text = " ".join([simple_stemmer(word) for word in text.split()])
                document_texts.append(stemmed_text)
                document_labels.append(os.path.basename(file[:-4]))
    
    # Return list of full nursery rhymes and list of the names of each file
    return document_texts, document_labels

Now lets load in our dataset of all of our nursery rhymes, here our path starts with `..`, which tells Python to move up one file level to find the folder `data` which then contains the folder `nursery-rhymes` which contains our dataset of text files.

In [45]:
folder_path = "../data/nursery-rhymes"
document_texts, document_labels = load_text_documents(folder_path)
print(f'loaded {len(document_labels)} documents')

loaded 298 documents


Lets print out the name and contents of the first file we have loaded to see it is working correctly:

In [46]:
print(f'The first document is {document_labels[0]}, which goes:')
print(document_texts[0])

The first document is five-little-ducks, which goes:
Five Little Ducks went out one day, over the hills and far away. Mother Duck said, “Quack, Quack, Quack, Quack,” but only four little ducks came back. Four little ducks went out one day, over the hills and far away Mother Duck said, “Quack, Quack, Quack, Quack,” but only three little ducks came back. (Repeat counting down to “but no little ducks came back.”) Sad mother duck went out one day, over the hills and far away Mother Duck said, “Quack, Quack, Quack, Quack,” and five little ducks came back.


Now lets define our set of stop words, these are words that we will remove from nursery rhymes that don't tell us anything meaningful about the contents of each individual rhyme. Often these are the most commonly used words like `and`, `it` and many others. 

Depending on what we are doing there may also be other stop words that we need to add that are specific to our dataset. To figure out what these are we need to have a good understanding of what is in the dataset to begin with.

Here we are defining our set of stop words manually as a list of words. We will be coming back to this cell in the task to expand our list of stop words.

<a id='stop-words'></a>

In [47]:
stop_words = ['and', 'it']

Now lets create our bag of words representation of our dataset. This time we will be looking at all of the words in the dataset, not just each individual file. We do this so we can make comparisons between the nursery rhymes (aka our documents).

We will have a table (aka matrix) that is 299 in length (the number of nursery rhymes in our dataset) by the number of words in our dataset. When you start working on the task you will notice that the number of words will change when you start adding more stop words an rules to the stemmer.

In [48]:
vectorizer = CountVectorizer(stop_words=stop_words)
bag_of_words = vectorizer.fit_transform(document_texts)
print(f'Our bag of words is a matrix of the shape and size {bag_of_words.shape}')

Our bag of words is a matrix of the shape and size (298, 3416)


If we look at  our bag_or_words representation we we can see that it is a big table showing the counts of each word with each nursery rhyme. You will notice that for many of these entries they are 0. As we have a large variety of words across our dataset, but each nursery rhyme only contains a limited vocabulary.

In [49]:
bow_df = pd.DataFrame(bag_of_words.toarray(), columns=vectorizer.get_feature_names_out(), index=document_labels)
bow_df

,15,1784,1812,3x,4th,aaples,abcs,abhors,aboard,about,...,yours,yourself,youth,yum,yummy,zany,zee,zero,zoo,zoom
five-little-ducks,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
old-mother-hubbard,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hey-diddle-diddle,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i-love-little-pussy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
skip-to-my-loo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
splash-splash,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
five-little-speckled-frogs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,10,0,0,0,0,0,0
here-i-am-little-jumping-joan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
twinkle-twinkle-little-star,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


If we want to see all of our words we can print them with the following code:


<a id='all-words'></a>

In [50]:
np.set_printoptions(threshold = np.inf)
print(vectorizer.get_feature_names_out())

['15' '1784' '1812' '3x' '4th' 'aaples' 'abcs' 'abhors' 'aboard' 'about'
 'above' 'accidentally' 'across' 'adde' 'added' 'additional' 'adieu'
 'adore' 'adored' 'adorned' 'afar' 'afeard' 'aff' 'afford' 'afore'
 'afraid' 'after' 'afternoon' 'again' 'against' 'age' 'ago' 'agreed' 'ah'
 'ahead' 'aiken' 'ain' 'air' 'airn' 'al' 'alabama' 'alas' 'alaska'
 'aldgate' 'ale' 'alehouse' 'alice' 'alive' 'all' 'alleluia' 'alligator'
 'allí' 'almost' 'alone' 'along' 'already' 'alternate' 'alternatively'
 'always' 'am' 'amazing' 'among' 'amor' 'ampersand' 'an' 'ancient' 'anew'
 'angel' 'angelic' 'angels' 'animal' 'ankle' 'ann' 'ano' 'another'
 'answer' 'ant' 'anthems' 'ants' 'anunciando' 'any' 'anymore' 'anyone'
 'anything' 'apart' 'appear' 'appearing' 'appetite' 'apple' 'apples'
 'april' 'aprill' 'aprons' 'are' 'arkan' 'arkansas' 'arm' 'arms' 'around'
 'arrow' 'arthur' 'as' 'ask' 'asked' 'asking' 'asks' 'asleep' 'ass'
 'astros' 'at' 'ate' 'attention' 'attitude' 'autumn' 'avenue' 'avignon'
 'avè' 'awa

To see the names of all of our nursery rhymes and their indexes in the documents lists we can run the following code. This will give us a [dictionary](https://www.w3schools.com/python/python_dictionaries.asp), a very useful data structure in Python, that we can use to find the index value of each nursery rhyme, using he nursery rhyme as our **keys**.

In [51]:
# This line of code is originally from: https://stackoverflow.com/a/36460020
document_labels_dict = {k: v for v, k in enumerate(document_labels)}
document_labels_dict

{'five-little-ducks': 0,
 'old-mother-hubbard': 1,
 'hey-diddle-diddle': 2,
 'i-love-little-pussy': 3,
 'skip-to-my-loo': 4,
 'jack-a-nory': 5,
 'baby-shark-dance': 6,
 'deck-the-halls': 7,
 'what-child-is-this': 8,
 'to-bed-to-bed': 9,
 'do-you-hear-what-i-hear': 10,
 'hes-got-the-whole-world-in-his-hands': 11,
 'head-shoulders-knees-and-toes': 12,
 'solomon-grundy': 13,
 'johny-johny-yes-papa': 14,
 'see-saw-margery-daw': 15,
 'jingle-bells': 16,
 'hickory-dickory-dock': 17,
 'autumn-leaves-are-falling-down': 18,
 'a-sailor-went-to-sea-sea-sea': 19,
 'mondays-child': 20,
 'pease-porridge-hot': 21,
 'bella-ciao': 22,
 'way-up-high-in-the-apple-tree': 23,
 'aiken-drum': 24,
 'feliz-navidad': 25,
 'lazy-mary': 26,
 'draw-a-pail-of-water': 27,
 'swing-low-sweet-chariot': 28,
 'here-we-go-looby-loo': 29,
 'going-on-a-bear-hunt': 30,
 'cobbler-cobbler-mend-my-shoe': 31,
 'a-tisket-a-tasket': 32,
 'turkey-in-the-straw': 33,
 'the-grand-old-duke-of-york': 34,
 'sur-le-pont-davignon': 35,
 'd

Now we can look at just the word counts for the first nursery rhyme (0), as we can see they are still mostly 0 entries.

In [52]:
single_row_df = bow_df.iloc[0]
single_row_df

15      0
1784    0
1812    0
3x      0
4th     0
       ..
zany    0
zee     0
zero    0
zoo     0
zoom    0
Name: five-little-ducks, Length: 3416, dtype: int64

To get a better sense of the contents of the rhymes, lets get rid of all of the counts that are 0. The simple way to do this is to replace them with the special variable `None` and then to remove of none-entries.

We can now get a much better sense of what our nuersery rhyme is about:

In [53]:
single_row_df = single_row_df.replace(0,None)
single_row_df = single_row_df.dropna()
single_row_df

away         3
back         4
but          3
came         4
counting     1
day          3
down         1
duck         4
ducks        6
far          3
five         2
four         2
hills        3
little       6
mother       4
no           1
one          3
only         2
out          3
over         3
quack       12
repeat       1
sad          1
said         3
the          3
three        1
to           1
went         3
Name: five-little-ducks, dtype: object

Now we are going to use our bag of words representation to do build a simple search engine based on key words. We will use the [dot product](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) to compare our poems with a representation of an individual word to see their similiarity. 

**Don't worry if you don't understand this code** for now. Linear algebra will be covered in more detail later in the term in the [STEM for Creatives](https://moodle.arts.ac.uk/course/view.php?id=79539) unit, and dictionaries and lambda functions will be also be covered later in STEM for Creative and Introduction to Data Science.

In [54]:
def retrieve_documents_by_keyword(document_labels, bow_matrix, vectorizer, keyword):
    keyword_bow = vectorizer.transform([keyword])
    
    # Use the dot product to find nursery rhymes that have our keyword 
    dot_product = np.dot(bow_matrix, keyword_bow.T).toarray().flatten()

    # Create a dictionary to store document names and their relevance scores then sort it
    document_relevance = {doc: score for doc, score in zip(document_labels, dot_product) if score > 0}
    sorted_documents = sorted(document_relevance.items(), key=lambda x: x[1], reverse=True)

    return sorted_documents

We can now use our function to find the most relevant nursery rhymes based on a key word, or multiple keywords. The document score tells us the relevance of each rhyme. Using our simple representation of text we have made a simple search engine!

Try a few different keywords to see what rhymes you can retrieve.

<a id='search'></a>

In [55]:
keyword = "sea"

# Retrieve documents containing the keyword
document_scores = retrieve_documents_by_keyword(document_labels, bag_of_words, vectorizer, keyword)
document_scores

[('theres-a-hole-in-the-bottom-of-the-sea', 24),
 ('a-sailor-went-to-sea-sea-sea', 12),
 ('wynken-blynken-and-nod', 5),
 ('my-bonnie-lies-over-the-ocean', 3),
 ('if-all-the-seas-were-one-sea', 3),
 ('do-you-hear-what-i-hear', 2),
 ('rub-a-dub-dub', 2),
 ('bobby-shafto', 1),
 ('three-wise-men-of-gotham', 1),
 ('i-had-a-little-nut-tree', 1),
 ('i-see-the-moon-and-the-moon-sees-me', 1),
 ('make-new-friends-but-keep-the-old', 1),
 ('if-all-the-world-was-apple-pie', 1),
 ('lift-every-voice-and-sing', 1),
 ('if-all-the-world-were-paper', 1)]

# Tasks

**Task 1:** Try some different keywords in the [search tool](#search), then look at the rhymes to see if you search results look reasonable. 

**Task 2:** Go to the cell that defines the [stop words](#stop-words) and add start adding more stop words to the set. Do this **without** using a preset library of English stop words. Try looking at the list of [all of the words](#all-words) in the dataset for examples of words that aren't specific to the rhymes.

**Task 3a:** Go to the cell that defines the [stemmer](#stemmer) and uncomment the first rule. The run all of the cells after that. See how that changes the number of words that are in our dataset.

**Task 3b:** Now try to define your own rules for the [stemmer](#stemmer). Try looking at the list of [all of the words](#all-words) in the dataset for words that are similiar but have different endings, that should give you some clues for what rules of letters to remove that you can add to the stemmer code.  

**Task 4:** Once you have build your stemmer and stop words database, go back to the [search tool](#search), are you still getting the results you got earlier. Has the effectiveness of the search improved? 

**Bonus task** If you have completed all of the tasks and want a further challenge, can you replace the stemmer with the [NLTK WordNet lemmatizer](https://pythonprogramming.net/lemmatizing-nltk-tutorial/)? How does that effect the search function compared to the stemmer?